In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
import pygentoolbox.Tools
import pandas as pd
import os
#dir(pygentoolbox.Tools)

In [133]:
import datetime
print(datetime.datetime.now())

2019-06-19 12:07:51.992010


In [140]:
cutoff = 0  # designate desired count cutoff, counts are number of reads mapped to a feature # keep counts greater than this value
verbose = False
f1 = 'D:\\LinuxShare\\Projects\\Theresa\\bedtools\\Intersect_MacAndIES_52Late_Piwi08'
f2 = 'D:\\LinuxShare\\Projects\\Theresa\\bedtools\\Intersect_MacAndIES_53Later_Piwi08'

df1 = pd.read_csv(f1, sep='\t', header=None) # index_col=0
df2 = pd.read_csv(f2, sep='\t', header=None) # index_col=0

df1 = df1.set_index(df1.iloc[:, 0] + '.' + df1.iloc[:, 3].map(str) + '.' + df1.iloc[:, 4].map(str)) # make unique indices
df2 = df2.set_index(df2.iloc[:, 0] + '.' + df2.iloc[:, 3].map(str) + '.' + df2.iloc[:, 4].map(str))

df1trim = df1[df1.iloc[:, 9] > cutoff]  # pygentoolbox.Tools.keep_rows_df_greater_than(df1, 0, 9)  # dataframe, cutoff value, column index to filter
df2trim = df2[df2.iloc[:, 9] > cutoff]  # pygentoolbox.Tools.keep_rows_df_greater_than(df2, 0, 9)  # dataframe, cutoff value, column index to filter
#print(df1trim.iloc[1,:])
#print(df1trim.head())
#print(df2trim.iloc[1,:])

dfinter1, dfinter2 = pygentoolbox.Tools.intersect_dfs_by_common_indices(df1trim, df2trim)  # intersection of two dfs indices

print(type(dfinter1))
print(type(dfinter2))
print(dfinter1.shape)
print(dfinter2.shape)

f= f1 + '.greater%d.inter' % (cutoff)
print('Output File: %s' % (f))
dfinter1.to_csv(f, sep='\t', index=False, header=None)

f = f2 + '.greater%d.inter' % (cutoff)
print('Output File: %s' % (f))
dfinter2.to_csv(f, sep='\t', index=False, header=None)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(147, 10)
(147, 10)
Output File: D:\LinuxShare\Projects\Theresa\bedtools\Intersect_MacAndIES_52Late_Piwi08.greater0.inter
Output File: D:\LinuxShare\Projects\Theresa\bedtools\Intersect_MacAndIES_53Later_Piwi08.greater0.inter


In [141]:
newdf = pd.concat([dfinter1, dfinter2.iloc[:,9]], axis=1)
print(newdf.head())
print(newdf.shape)

p, filename = os.path.split(f1)  # get the folder path
f = os.path.join(p,'Counts_greaterthan%d_inter_52LateVs53Later.tsv' % (cutoff))  # make full path with filename
print('Output File: %s' % (f))
newdf.to_csv(f, sep='\t', index=False, header=None)

                                                             0     1  \
scaffold51_100_with_IES.143972.144106  scaffold51_100_with_IES  MICA   
scaffold51_102_with_IES.153433.153941  scaffold51_102_with_IES  MICA   
scaffold51_102_with_IES.330974.332147  scaffold51_102_with_IES  MICA   
scaffold51_103_with_IES.150661.150734  scaffold51_103_with_IES  MICA   
scaffold51_103_with_IES.233118.233236  scaffold51_103_with_IES  MICA   

                                                                  2       3  \
scaffold51_100_with_IES.143972.144106  internal_eliminated_sequence  143972   
scaffold51_102_with_IES.153433.153941  internal_eliminated_sequence  153433   
scaffold51_102_with_IES.330974.332147  internal_eliminated_sequence  330974   
scaffold51_103_with_IES.150661.150734  internal_eliminated_sequence  150661   
scaffold51_103_with_IES.233118.233236  internal_eliminated_sequence  233118   

                                            4  5  6  7  \
scaffold51_100_with_IES.143972.144

In [142]:
matdf = pd.concat([dfinter1.iloc[:,9], dfinter2.iloc[:,9]], axis=1)
print(matdf.shape)
print(matdf.head(10))

dfnorm = pygentoolbox.Tools.median_of_ratios(matdf, verbose)
print(dfnorm.shape)
print(dfnorm.head(10))

p, filename = os.path.split(f1)  # get the folder path
f = os.path.join(p, 'Counts_MedianOfRatiosNormalized_greaterthan%d_inter_52LateVs53Later.tsv' % (cutoff))  # make full path with filename
print('Output File: %s' % (f))
dfnorm.to_csv(f, sep='\t', index=True, header=None)

(147, 2)
                                        9   9
scaffold51_100_with_IES.143972.144106   2   1
scaffold51_102_with_IES.153433.153941   2   6
scaffold51_102_with_IES.330974.332147  21  42
scaffold51_103_with_IES.150661.150734   2   2
scaffold51_103_with_IES.233118.233236   1   1
scaffold51_103_with_IES.300906.300991   3   1
scaffold51_105_with_IES.251749.252012   2   1
scaffold51_105_with_IES.256049.258531   2   4
scaffold51_107_with_IES.49046.49124     1   1
scaffold51_107_with_IES.218403.218428   1   1
(147, 2)
                                          9     9
scaffold51_100_with_IES.143972.144106   2.0   1.0
scaffold51_102_with_IES.153433.153941   2.0   6.0
scaffold51_102_with_IES.330974.332147  21.0  42.0
scaffold51_103_with_IES.150661.150734   2.0   2.0
scaffold51_103_with_IES.233118.233236   1.0   1.0
scaffold51_103_with_IES.300906.300991   3.0   1.0
scaffold51_105_with_IES.251749.252012   2.0   1.0
scaffold51_105_with_IES.256049.258531   2.0   4.0
scaffold51_107_with_IES.49

In [1]:
#####  This output reports the same IES that have counts over threshold in late and later #####